In [ ]:
# Instalar as bibliotecas
!pip install pandas
!pip install nltk
!pip install spacy
!python -m spacy download pt_core_news_sm

# Carregar as bibliotecas
import pandas as pd
import spacy
import re
import unicodedata

In [ ]:
# Importando a library Natural Language Toolkit - NLTK para tratamento de linguagem natural
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams

# Importar as stopwords
nltk.download('stopwords') # mudei essa pra portugues
from nltk.corpus import stopwords
stopwords = set(stopwords.words('portuguese'))

In [ ]:
# Abrindo e lendo o arquivo JSON
# Pressupondo que o arquivo JSON tem uma estrutura em que os textos estão em uma coluna chamada "TRANSCRITO".
# Caso o nome ou estrutura seja diferente, ajuste os campos abaixo.

# Carregar o arquivo JSON
file_path = '/content/unified_data.json'  # Altere o caminho conforme necessário
df = pd.read_json(file_path)

# Verificar as primeiras linhas para confirmar os dados
df.head()

In [ ]:
# Lematizando o texto
# Carregando o modelo de linguagem spaCy na língua portuguesa
nlp = spacy.load('pt_core_news_sm')

# Função para lematizar o texto
def lemmatize_text(text):
    doc = nlp(text)  # Processa o texto com spaCy
    return ' '.join([token.lemma_ for token in doc])  # Retorna o texto lematizado

# Aplica a função de lematização à coluna do corpus textual
df['Transcrito_Lematizado'] = df['content'].apply(lemmatize_text)

# Exibir os resultados com a coluna lematizada
df.head()

In [ ]:
# Função para retirar caracteres não ASCII
def remove_non_ascii(word):
    new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_word

# Função para processar cada célula individualmente
def process_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r"\'", "", text)
    text = text.lower()
    words = text.split()
    words = [remove_non_ascii(word) for word in words]
    return ' '.join(words)

# Aplicar essa função a todas as células
df['Transcrito_Processado'] = df['Transcrito_Lematizado'].apply(process_text)

# Exibir os resultados com a coluna limpa
df.head()

In [ ]:
# Removendo as stopwords
def remove_stopwords_and_non_ascii(text):
    words = text.split()
    words_without_stopwords = [word for word in words if word not in stopwords]
    cleaned_text = ' '.join(words_without_stopwords)
    return cleaned_text

# Aplicando a função remove_stopwords_and_non_ascii à coluna 'Transcrito_Processado'
df['Transcrito_Limpo'] = df['Transcrito_Processado'].apply(remove_stopwords_and_non_ascii)

# Exibir os resultados
df.head()

In [ ]:
from functools import wraps
import time
import spacy
import string

def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"{func.__name__} took {end - start:.4f} seconds")
        return result
    return wrapper

@timeit
def clean_text(text):
    # Load the Portuguese language model
    nlp = spacy.load('pt_core_news_sm')

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation (including quotation marks, periods, commas)
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Process the text
    doc = nlp(text)

    # Define sets of parts of speech to remove
    remove_pos = {
        'NUM',      # Numbers
        'ADP',      # Prepositions
        'DET',      # Determiners (articles)
        'CCONJ',    # Coordinating conjunctions
        'SCONJ',    # Subordinating conjunctions
        'PRON',     # Pronouns
    }

    # Common verbs to remove (you can expand this list)
    common_verbs = {
        'ser', 'estar', 'ter', 'haver', 'ir', 'poder',
        'dever', 'ficar', 'fazer', 'dizer', 'dar',
        'ver', 'vir', 'querer', 'saber', 'chegar', 'partir'
    }

    # Filter out unwanted tokens
    filtered_tokens = [
        token.text for token in doc
        if token.pos_ not in remove_pos
        and token.lemma_.lower() not in common_verbs
        and token.is_alpha  # Ensure only alphabetic tokens remain
    ]

    # Join the filtered tokens back into a sentence
    return ' '.join(filtered_tokens)

# Example usage for a DataFrame
#df['Transcrito_Limpo2'] = df['Transcrito_Limpo'].apply(clean_text)
content_tokenized_listed = df['Transcrito_Limpo'].to_list()
example = content_tokenized_listed[0]
clean_text(example)

df2 = df.copy().iloc[:10]
df2['Transcrito_Limpo'].apply(clean_text)

# Salvando o arquivo
# Salvar no formato Excel
df.to_excel('noticiasptexpandido_processado.xlsx', index=False)

In [ ]:
# Function to remove non-word characters and specified words
def clean_text(text):
    # Remove non-word characters
    text = re.sub(r'[^\w\s]', '', text)
    # Remove specified words
    words_to_remove = #inserir aqui as palavras
    text = ' '.join([word for word in text.split() if word not in words_to_remove])
    return text

# Apply the cleaning function to the 'Transcrito_Limpo' column
df['Transcrito_Limpo'] = df['Transcrito_Limpo'].apply(clean_text)

# Save the modified DataFrame to a new Excel file
df.to_excel('noticiaspt_processado_cleaned.xlsx', index=False)

# Download the cleaned file
files.download('noticiaspt_processado_cleaned.xlsx')